In [17]:
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import json
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input, InputLayer
from tensorflow.keras import optimizers
import tensorflow.keras.backend as K
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import nasnet, resnet50, densenet
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

%matplotlib inline

In [18]:
df = pd.read_csv('Kaggle2/train.csv')
df.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


In [23]:
dict_columns = ['belongs_to_collection', 'genres', 'production_companies',
                'production_countries', 'spoken_languages', 'Keywords', 'cast', 'crew']

def text_to_dict(df):
    for column in dict_columns:
        df[column] = df[column].apply(lambda x: {} if pd.isna(x) else ast.literal_eval(x) )
    return df
        
dfx = text_to_dict(df)
for col in dict_columns:
       df[col]=dfx[col]

In [36]:
df['production_companies'].apply(lambda x:len(x) if x!= {} else 0).head()

0    3
1    1
2    3
3    0
4    0
Name: production_companies, dtype: int64

In [37]:
collections=df['production_companies'].apply(lambda x : x[0]['name'] if x!= {} else '?')
collections

0                            Paramount Pictures
1                          Walt Disney Pictures
2                                    Bold Films
3                                             ?
4                                             ?
5                                             ?
6                          Ghost House Pictures
7                                             ?
8                          Walt Disney Pictures
9                     Castle Rock Entertainment
10                               United Artists
11       Twentieth Century Fox Film Corporation
12                               DreamWorks SKG
13                         Double Feature Films
14                               DreamWorks SKG
15                                   Hypnopolis
16                               DreamWorks SKG
17                                    BBC Films
18                              UK Film Council
19       Twentieth Century Fox Film Corporation
20                              New Line

In [39]:
collections=df['production_companies'].apply(lambda x : x[0]['name'] if x!= {} else '?')
collections

0                            Paramount Pictures
1                          Walt Disney Pictures
2                                    Bold Films
3                                             ?
4                                             ?
5                                             ?
6                          Ghost House Pictures
7                                             ?
8                          Walt Disney Pictures
9                     Castle Rock Entertainment
10                               United Artists
11       Twentieth Century Fox Film Corporation
12                               DreamWorks SKG
13                         Double Feature Films
14                               DreamWorks SKG
15                                   Hypnopolis
16                               DreamWorks SKG
17                                    BBC Films
18                              UK Film Council
19       Twentieth Century Fox Film Corporation
20                              New Line

In [19]:
print('Number of genres in films')
df['genres'].apply(lambda x: len(x) if x != {} else 0).value_counts()

Number of genres in films


TypeError: object of type 'float' has no len()

In [20]:
missing_data = df.isnull()

In [21]:
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")

id
False    3000
Name: id, dtype: int64

belongs_to_collection
True     2396
False     604
Name: belongs_to_collection, dtype: int64

budget
False    3000
Name: budget, dtype: int64

genres
False    2993
True        7
Name: genres, dtype: int64

homepage
True     2054
False     946
Name: homepage, dtype: int64

imdb_id
False    3000
Name: imdb_id, dtype: int64

original_language
False    3000
Name: original_language, dtype: int64

original_title
False    3000
Name: original_title, dtype: int64

overview
False    2992
True        8
Name: overview, dtype: int64

popularity
False    3000
Name: popularity, dtype: int64

poster_path
False    2999
True        1
Name: poster_path, dtype: int64

production_companies
False    2844
True      156
Name: production_companies, dtype: int64

production_countries
False    2945
True       55
Name: production_countries, dtype: int64

release_date
False    3000
Name: release_date, dtype: int64

runtime
False    2998
True        2
Name: runtime, dtype: in

In [7]:
df.dropna(subset=["genres"], axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [8]:
df = df[['popularity','release_2000', 'budget', 'genres', 'revenue']]
df.head(15)

KeyError: "['release_2000'] not in index"

In [ ]:
df.describe().T

In [ ]:
zero_data = df['budget'].split('''''')
zero_data = df['production_companies'].str.split(":", n = 1, expand = True) 

In [ ]:
#print (zero_data.value_counts())